#  <ins>Verkehrsuntersuchung</ins>

## Initialisierung

In [1]:
import gzip
import numpy as np
import pandas as pd
from sqlalchemy import create_engine#, text, MetaData, Table, Column, String
from geopy.geocoders import Nominatim

In [2]:
sql_password = '' # Benutze hier dein MySQL- Passwort
engine = create_engine('mysql+mysqlconnector://root:' + sql_password + '@localhost:3306/verkehrsprojekt')
connection = engine.connect()

## Datenuntersuchung

### PKW- Daten

Quelle: https://api.viz.berlin.de/daten/verkehrsdetektion?path=2018%2FMessquerschnitt+%28fahrtrichtungsbezogen%29%2F <br>
Die Quelle ist für 2018. Selbiges gibt es auch für die anderen Jahre.

##### Erstelle für jeden Monat/Jahr ein Dataframe zur Untersuchung. Sammle all diese Dataframes in df_collection

In [3]:
def csvgz_to_df(new_csvgz, start_date, end_date):
    with gzip.open(new_csvgz, 'rt') as f:
        data_temp = pd.read_csv(f, sep=';')
    
    # Einstellen, welche Spalten wir brauchen
    data_temp['stunde'] = data_temp['stunde'].apply(lambda x: str(x) if len(str(x)) == 2 else '0'+str(x))
    data_temp['timestamp'] = pd.to_datetime(data_temp['tag'] + '-' + data_temp['stunde'], format = '%d.%m.%Y-%H' )
    data_temp = data_temp[['timestamp','mq_name','q_pkw_mq_hr']]

    # Erstelle ein Dataframe, das jedes TE als eigene Spalte hat
    station_names = pd.unique(data_temp['mq_name'])
    reference_timestamps = pd.date_range(start=start_date, end=end_date, freq='h')
    data = pd.DataFrame({'timestamp': reference_timestamps})
    data = data.set_index('timestamp')
    for string in station_names:
        filt = ( data_temp['mq_name'] == string )
        data_temp_te = data_temp[filt]
        data_temp_te = data_temp_te.set_index('timestamp')
        data_temp_timestamps = data_temp_te.index
        present_timestamps = data_temp_timestamps[data_temp_timestamps.isin(reference_timestamps)]
        data[string] = np.nan
        data.loc[present_timestamps, string] = data_temp_te.loc[present_timestamps,'q_pkw_mq_hr']
        data = data.copy() # For performance improvement
        if string[-1] == 'n': # Manche Stationsnamen ab 01.2023 haben ein 'n' am Ende. Wir entfernen das.
            data = data.rename(columns = {string:string[:-1]})
            if (data.columns == string[:-1]).sum() > 1: # Manche Spaltennamen kommen jetzt doppelt vor, zum Beispiel 'TE134' in 01.2023. Dann nehmen wir den Mittelwert aus den doppelten Spalten.
                column_temp = data[string[:-1]].mean(axis = 1)
                data = data.drop(columns = string[:-1])
                data[string[:-1]] = column_temp    
                
    # Behalte nur Zeilen, in denen ein Minimum von nicht-NaN- Werten stehen
    minimum_notnas_per_row = 25
    filt = ( data.notna().sum(axis=1) >= minimum_notnas_per_row )
    data = data[filt]
    
    data['Durchschnitt'] = data.mean(axis = 1)
    return data

In [4]:
df_collection = []
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for year in range(2018, 2023 +1):
    for month in months:
        new_csvgz = 'mq_hr_' + str(year) + '_' + month  + '.csv.gz'
        start_date = pd.Timestamp(year=year, month=int(month), day=1, hour=0, minute=0, second=0)#str(year) + '-' + month + '-01 00:00:00'
        end_date = pd.Timestamp(year=year, month=int(month), day=start_date.days_in_month, hour=23, minute=0, second=0)#str(year) + '-' + month + '-01 00:00:00'
        data = csvgz_to_df(new_csvgz, start_date, end_date)
        df_collection.append(data)

#### Excel- Tabelle vervollständigen

In [5]:
def get_district_and_neighborhood(lat, lon):
    # Initialisiere den Nominatim-Geocoder
    geolocator = Nominatim(user_agent = "berlin_geocoder_2345671234") # can use any user_agent name as far as I've understood it.
    
    # Koordinaten umkehren (Reverse Geocoding)
    location = geolocator.reverse((lat, lon), exactly_one=True)
    
    if location and location.raw:
        # Extrahiere die relevanten Informationen
        address = location.raw.get('address', {})
        district = address.get('suburb', 'Bezirk nicht gefunden')
        borough = address.get('borough', 'Stadtteil nicht gefunden')
        return district, borough
    else:
        return None, None

def district_neighborhood_pkw(row):
    lat, lon = row[['BREITE (WGS84)','LÄNGE (WGS84)']]
    district, neighborhood = get_district_and_neighborhood(lat, lon)
    return district, neighborhood

In [9]:
# Erstelle die PKW- Stammdaten- Excel mitsamt Bezirken und Stadtteilen
df = pd.read_excel('Stammdaten_Verkehrsdetektion_2022_07_20.xlsx')
df_sample = df[['LÄNGE (WGS84)','BREITE (WGS84)']]
new_column_series = df_sample.apply(district_neighborhood_pkw, axis = 1)
new_columns = new_column_series.apply(pd.Series)
df[['Bezirk','Stadtteil']] = new_columns
df.to_excel('Stammdaten_pkw.xlsx', index = False)

#### Eintragen in eine SQL- Datenbank

In [10]:
big_df = pd.DataFrame()
for data in df_collection:
    big_df = pd.concat([big_df,data])
big_df = big_df.sort_index(axis = 1)

# Stichwort Chunking: Die Tabelle ist so groß, dass wir sie in zwei Teilen in die Datenbank laden müssen. Sonst kommen komische Fehlermeldungen :(((((
# Part 1
big_df_part_1 = big_df.head( big_df.shape[0] // 2 )
big_df_part_1.to_sql('pkw_daten', engine, if_exists = 'replace', index = True) # Ich betone if_exists = 'replace'
# Part 2
big_df_part_2 = big_df.tail( big_df.shape[0] - (big_df.shape[0] // 2) )
big_df_part_2.to_sql('pkw_daten', engine, if_exists = 'append', index = True) # Hier hingegen if_exists = 'append'

25970

### Fahrrad- Daten 

#### Fahrrad- Daten einpflegen

Quelle: https://daten.berlin.de/datensaetze/radzahldaten-in-berlin

In [13]:
# Für 'Fahrraddaten.xlsx' auf dem sheet 'Standortdaten', erstelle die Spalten 'Bezirk' und 'Stadtteil'

def district_neighborhood_fahrrad(row):
    lat, lon = row[['Breitengrad','Längengrad']]
    district, neighborhood = get_district_and_neighborhood(lat, lon)
    return district, neighborhood

if False:
    df = pd.read_excel('gesamtdatei-stundenwerte.xlsx', sheet_name = 'Standortdaten')
    df_sample = df[['Breitengrad','Längengrad']]
    new_column_series = df_sample.apply(district_neighborhood_fahrrad, axis = 1)
    new_columns = new_column_series.apply(pd.Series)
    df[['Stadtteil','Bezirk']] = new_columns
    df.to_excel('Fahrrad-Stammdaten.xlsx', index = False)

In [16]:
if False:
    # Erstelle für jedes sheet (d.h. jedes Jahr) einen Dataframe und konkatiniere diese Dataframes zu einem df_big
    for year in range(2018,2023 +1):
        globals()['df_' + str(year)] = pd.read_excel('gesamtdatei-stundenwerte.xlsx', sheet_name = 'Jahresdatei ' + str(year))
        globals()['df_' + str(year)] = globals()['df_' + str(year)].rename(columns = {'Zählstelle        Inbetriebnahme':'timestamp'})
    df_list = [globals()['df_' + str(year)] for year in range(2018,2023 +1)]
    df_big = pd.concat(df_list)
    df_big = df_big.reset_index(drop = True)

    # Entferne das Datum aus den Spaltennamen. z.B.  '02-MI-JAN-N 01.04.2015' --> '02-MI-JAN-N'
    new_col_names = [string[:-11].strip() for string in df_big.columns[1:]]
    df_big.columns = pd.core.indexes.base.Index( list(df_big.columns[:1]) + new_col_names )

    #index setzen
    df_big = df_big.set_index('timestamp')

    #Füge eine Durchschnittsspalte ein
    df_big['Durchschnitt'] = df_big.mean(axis = 1)
    
    df_big.to_sql('fahrraddaten', engine, if_exists = 'replace')

In [17]:
if False:
    start = pd.to_datetime('2018-01-01 00:00:00')
    end = pd.to_datetime('2023-12-31 23:00:00')
    expected_timestamps = pd.date_range(start=start, end=end, freq='h')
    
    data = df_big[['Durchschnitt']]
    
    # Schritt 1: Überprüfen, ob alle erwarteten Timestamps im DataFrame vorhanden sind
    missing_timestamps = expected_timestamps.difference(data.index)
    
    # Schritt 2: Überprüfen, ob duplikate Werte vorhanden sind
    duplicate_timestamps = data.index[data.index.duplicated()]
    
    # Ausgabe:
    if missing_timestamps.empty and duplicate_timestamps.empty:
        print("Alle Stunden im Zeitraum sind genau einmal vorhanden.")
    else:
        if not missing_timestamps.empty:
            print(f"Fehlende Timestamps: {missing_timestamps}")
        if not duplicate_timestamps.empty:
            print(f"Duplizierte Timestamps: {duplicate_timestamps}")

Fehlende Timestamps: DatetimeIndex(['2019-03-31 02:00:00'], dtype='datetime64[ns]', freq='h')
